# Define tool and model of the tool

In [1]:
import sys

TOOLS_NAME_NER = "ner"
MODEL_TOOLS_NAME_NER = "ageng-anugrah/indobert-large-p2-finetuned-ner"

TOOLS_NAME_POS = "token-classification"
MODEL_TOOLS_NAME_POS = "ageng-anugrah/indobert-large-p2-finetuned-chunking"

MODEL_SIMILARITY_NAME = "paraphrase-multilingual-mpnet-base-v2"

# SAMPLE = sys.maxsize
SAMPLE = 20

# Import anything

In [2]:
import transformers
import evaluate
import torch
import operator
import re
import sys
import collections
import string
import contextlib
import gc
import random
import string

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from datetime import datetime
from huggingface_hub import notebook_login
from tqdm import tqdm
from huggingface_hub import HfApi
from sentence_transformers import SentenceTransformer, util

from datasets import (
    load_dataset, 
    Dataset,
    DatasetDict
)
from transformers import (
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EarlyStoppingCallback, 
    AutoModelForQuestionAnswering,
    AutoModelForTokenClassification,
    pipeline
)

# Retrieve QA dataset

In [3]:
conhelps = NusantaraConfigHelper()
data_qas = conhelps.filtered(lambda x: 'idk_mrc' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_qas['train'])
df_validation = pd.DataFrame(data_qas['validation'])
df_test = pd.DataFrame(data_qas['test'])

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_train['context']))):
    for j in df_train["qas"][i]:
        if len(j['answers']) != 0:
            new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                                'question': j['question'], 
                                                'answer': {"text": j['answers'][0]['text'], 
                                                           "answer_start": j['answers'][0]['answer_start'], 
                                                           "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, 
                                                           ignore_index=True)
        else:
            new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                                'question': j['question'], 
                                                'answer': {"text": str(), 
                                                           "answer_start": 0, 
                                                           "answer_end": 0}}, 
                                                           ignore_index=True)

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_validation['context']))):
    for j in df_validation["qas"][i]:
        if len(j['answers']) != 0:
            new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": j['answers'][0]['text'], 
                                                       "answer_start": j['answers'][0]['answer_start'], 
                                                       "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, 
                                                       ignore_index=True)
        else:
            new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": str(), 
                                                       "answer_start": 0, 
                                                       "answer_end": 0}}, 
                                                       ignore_index=True)        

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_test['context']))):
    for j in df_test["qas"][i]:
        if len(j['answers']) != 0:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": j['answers'][0]['text'], 
                                                       "answer_start": j['answers'][0]['answer_start'], 
                                                       "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, 
                                                       ignore_index=True)
        else:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": str(), 
                                                       "answer_start": 0, 
                                                       "answer_end": 0}}, 
                                                       ignore_index=True)

train_dataset = Dataset.from_dict(new_df_train)
validation_dataset = Dataset.from_dict(new_df_val)
test_dataset = Dataset.from_dict(new_df_test)

data_qas = DatasetDict({"train": train_dataset, "validation": validation_dataset, "test": test_dataset})
data_qas

  0%|          | 0/3 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 378/378 [00:01<00:00, 256.18it/s]


DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 9332
    })
    validation: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 764
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 844
    })
})

# Convert to NLI, with hypothesis being just do concat question & answer

## Convert Dataset to DataFrame format

In [4]:
seed_value = 42
random.seed(seed_value)

In [5]:
data_qas_train_df = (pd.DataFrame(data_qas["train"])).sample(n=SAMPLE, random_state=42)
data_qas_val_df = (pd.DataFrame(data_qas["validation"])).sample(n=SAMPLE, random_state=42)
data_qas_test_df = (pd.DataFrame(data_qas["test"])).sample(n=SAMPLE, random_state=42)

data_qas_train_df = data_qas_train_df.reset_index(drop=True)
data_qas_val_df = data_qas_val_df.reset_index(drop=True)
data_qas_test_df = data_qas_test_df.reset_index(drop=True)

## Retrieve answer text only

In [6]:
def retrieve_answer_text(data):
    for i in range(len(data)):
        data['answer'][i] = data['answer'][i]['text']
    return data

In [7]:
data_qas_train_df = retrieve_answer_text(data_qas_train_df)
data_qas_val_df = retrieve_answer_text(data_qas_val_df)
data_qas_test_df = retrieve_answer_text(data_qas_test_df)

## Delete all unanswerable row

In [8]:
data_qas_train_df = data_qas_train_df[data_qas_train_df['answer'] != '']
data_qas_val_df = data_qas_val_df[data_qas_val_df['answer'] != '']
data_qas_test_df = data_qas_test_df[data_qas_test_df['answer'] != '']

### Reset index number

In [9]:
data_qas_train_df = data_qas_train_df.reset_index(drop=True)
data_qas_val_df = data_qas_val_df.reset_index(drop=True)
data_qas_test_df = data_qas_test_df.reset_index(drop=True)

## Create NLI dataset from copy of QA dataset above

In [10]:
data_nli_train_df = data_qas_train_df.copy()
data_nli_val_df = data_qas_val_df.copy()
data_nli_test_df = data_qas_test_df.copy()

In [11]:
#data_nli_wrong_train_df = data_qas_train_df.copy()
#data_nli_wrong_val_df = data_qas_val_df.copy()
#data_nli_wrong_test_df = data_qas_test_df.copy()

## Convert context pair to premise (only renaming column)

In [12]:
data_nli_train_df = data_nli_train_df.rename(columns={"context": "premise"})
data_nli_val_df = data_nli_val_df.rename(columns={"context": "premise"})
data_nli_test_df = data_nli_test_df.rename(columns={"context": "premise"})

# Add contradiction label cases

## Import pipeline to create contradiction cases

In [13]:
nlp_tools_ner = pipeline(task = TOOLS_NAME_NER, 
                     model = MODEL_TOOLS_NAME_NER, 
                     tokenizer = AutoTokenizer.from_pretrained(MODEL_TOOLS_NAME_NER, 
                                                               model_max_length=512, 
                                                               truncation=True),
                     aggregation_strategy = 'simple')

In [14]:
nlp_tools_chunking = pipeline(task = TOOLS_NAME_POS, 
                     model = MODEL_TOOLS_NAME_POS, 
                     tokenizer = AutoTokenizer.from_pretrained(MODEL_TOOLS_NAME_POS, 
                                                               model_max_length=512, 
                                                               truncation=True),
                     aggregation_strategy = 'simple')

## Add NER and chunking tag column in DataFrame

In [15]:
def add_row_tag(answer, tag, ner=nlp_tools_ner, chunking=nlp_tools_chunking):

    if tag == "ner": tools=ner
    else: tools=chunking

    try:
        tag_answer = (tools(answer)[0]['entity_group'], answer)
    except:
        tag_answer = ("NULL", answer)
        
    return tag_answer

In [16]:
def add_premise_tag(data, tag, index, premise_array, ner=nlp_tools_ner, chunking=nlp_tools_chunking):

    if tag == "ner": tools=ner
    else: tools=chunking
    
    if len(tools(data['premise'][index])) == 0:
        premise_array.append("NO TOKEN DETECTED")
    
    else:
        for j in tools(data['premise'][index]):
            tag_premise = (j['entity_group'], j['word'])
            premise_array.append(tag_premise)

    return premise_array

In [17]:
def add_ner_and_chunking_all_tag(data):
    
    data['ner_tag_answer'] = ""
    data['chunking_tag_answer'] = ""
    
    data['ner_tag_premise'] = ""
    data['chunking_tag_premise'] = ""
    
    for i in tqdm(range(len(data))):
        
        answer = data['answer'][i]
        premise = data['premise'][i]
        
        ner_premise_array = []
        chunking_premise_array = []
            
        data['ner_tag_answer'][i] = add_row_tag(answer, "ner")
        data['chunking_tag_answer'][i] = add_row_tag(answer, "chunking")
                                                
        data['ner_tag_premise'][i] = add_premise_tag(data, "ner", i, ner_premise_array)
        data['chunking_tag_premise'][i] = add_premise_tag(data, "chunking", i, chunking_premise_array)  
    
    return data

In [18]:
data_nli_train_df = add_ner_and_chunking_all_tag(data_nli_train_df)
data_nli_val_df = add_ner_and_chunking_all_tag(data_nli_val_df)
data_nli_test_df = add_ner_and_chunking_all_tag(data_nli_test_df)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:32<00:00,  3.24s/it]


100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 125829.12it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 78545.02it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 52265.47it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 165732.38it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 54947.21it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13530.01it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 281182.39it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 499983.26it/s]

100%|███████████████████

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 51781.53it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 224981.82it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 95016.60it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 35345.26it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16777.22it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12446.01it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 105517.08it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 74235.47it/s]

100%|███████████████████

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 108162.57it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 149796.57it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 148617.07it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 114390.11it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 148617.07it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 116869.15it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 233016.89it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 150746.66it/s]

100%|███████████████████

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 141699.46it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 149796.57it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 164482.51it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 301465.60it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 193324.63it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 153125.38it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 200791.15it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 224123.11it/s]

100%|███████████████████

...

100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 317750.30it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 233016.89it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 48489.06it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 50331.65it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 200924.74it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 12945.38it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 136098.36it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 111682.65it/s]

100%|███████████████████

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 84126.44it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 134756.76it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 56375.05it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 21024.08it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 34663.67it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 22878.02it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 37449.14it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 22671.91it/s]

100%|███████████████████

100%|█████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 105149.68it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 116508.44it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 84855.86it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 119350.11it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 124407.32it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 50840.05it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 93503.59it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 68920.49it/s]

100%|███████████████████

100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 245760.00it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 120065.95it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 80942.71it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 69615.00it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 142179.80it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 209715.20it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 292625.86it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 122164.19it/s]

100%|███████████████████

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 72817.78it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 90394.48it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 69442.12it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 193397.30it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 104335.92it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 95979.50it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 137313.52it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 101400.76it/s]

100%|███████████████████

# Create wrong answer

This is the flow to create wrong answer:

1. Check the NER and POS/Chunking labels of the right_answer and context/premise.

2. Search and group NER and POS/Chunking labels that match the right_answer throughout the context/premise.

3. Perform NER classification. There will be two branches here, namely:

   3a. If the NER of the right_answer can be detected, then calculate the distance using semantic similarity or word vectors between the right_answer and various possible wrong_answers with the same NER as the right_answer. Once done, proceed to the final wrong_answer.
   
   3b. If the NER of the right_answer cannot be detected (NULL) or context/premise does not contain any of NER of right_answer, then the POS/Chunking of the right_answer will be identified.
   
4. Perform POS/Chunking classification. Continuation from point 3b. There will be two more branches:

   4a. If the POS/Chunking of the right_answer can be detected, then calculate the distance using semantic similarity or word vectors between the right_answer and various possible wrong_answers with the same POS/Chunking as the right_answer. Once done, proceed to the final wrong_answer.
   
   4b. If the POS/Chunking of the right_answer cannot be detected (NULL) or context/premise does not contain any of NER of right_answer, then the final wrong_answer will be chosen based on a random word (random_word) from the context/premise.

In [19]:
model_similarity = SentenceTransformer(MODEL_SIMILARITY_NAME)

def return_similarity_sorted_array(right_answer, sentence_array, model=model_similarity):
    
    embedding_right_answer = model.encode([right_answer], convert_to_tensor=True)
    embedding_sentence_array = model.encode(sentence_array, convert_to_tensor=True)
    
    cosine_scores = util.pytorch_cos_sim(embedding_right_answer, embedding_sentence_array)
    
    sorted_indices = cosine_scores.argsort(descending=True)[0]
    sorted_array = [sentence_array[i] for i in sorted_indices]
    
    return sorted_array

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [20]:
def remove_values_with_hash(arr):
    return [item for item in arr if "#" not in item]

In [21]:
def select_random_word(text):
    words = re.findall(r'\w+', text)
    random_word = random.choice(words)
    return random_word

In [22]:
def grouping_same_tag(tag_answer, tag_premise, same_tag_array):
    
    for tag in tag_premise:

        # Check is it in tuple?
        if isinstance(tag, tuple):
            tag_word = tag[0]
        else:
            tag_word = None
        
        if tag_answer == tag_word:
            same_tag_array.append(tag[1])

    return remove_values_with_hash(same_tag_array)

In [23]:
def sorting_similarity(data, right_answer, index, tag, plausible_answer_array):

    if tag == "ner": slice='same_ner_tag_answer'
    elif tag == "chunking": slice='same_chunking_tag_answer'
    else: slice=None

    # Find all the sorted (by similarity) plausible wrong answer, 
    # and remove hask & punctuation only answer
    if slice != None:
        wrong_answer_array = return_similarity_sorted_array(right_answer, data[slice][index])
    else:
        wrong_answer_array = return_similarity_sorted_array(right_answer, plausible_answer_array)
    
    plausible_answer_array = remove_values_with_hash(wrong_answer_array)
    plausible_answer_array = [string for string in plausible_answer_array \
                                      if not contains_only_punctuation(string)]

    # Only return the most similar to right_answer
    wrong_answer = plausible_answer_array[0]
    
    assert isinstance(wrong_answer, str)
    assert isinstance(plausible_answer_array, list)
    
    return wrong_answer, plausible_answer_array

In [24]:
def find_substring_span(long_string, substring):
    long_string = long_string.lower()
    substring = substring.lower()
    
    start_index = long_string.find(substring)
    
    if start_index != -1:
        end_index = start_index + len(substring) - 1
        return start_index, end_index
    else:
        return None

In [25]:
def check_span_overlap(span1, span2):
    return span1[0] <= span2[1] and span2[0] <= span1[1]

def check_string_overlap(str1, str2):
    return (str1[-1] >= str2[0]) \
            or (str1 in str2) \
            or (str2 in str1)

def contains_only_punctuation(text):
    return all(char in string.punctuation for char in text)

In [26]:
def replace_same_answer(right_answer, 
                        wrong_answer, 
                        premise, 
                        plausible_answer_array):
    
    # Removing right answer & wrong answer in this particular time
    plausible_answer_array = [item for item in plausible_answer_array \
                              if item not in [right_answer, wrong_answer]]

    if len(plausible_answer_array) <= 1:
        wrong_answer = select_random_word(premise)
        properties = """Detected span that is the SAME as the right answer, 
                                search random word from premise"""

    else:
        wrong_answer = plausible_answer_array[0] # Take the highest value in the sorted array
        properties = """Detected span that is the SAME as the right answer, 
                                search the highest value in the sorted array"""

    return wrong_answer, properties, plausible_answer_array

In [27]:
def create_wrong_answer(data):
    
    data['same_ner_tag_answer'] = ""
    data['same_chunking_tag_answer'] = ""
    data['wrong_answer'] = ""
    data['plausible_answer_based_on_method'] = ""
    data['properties'] = ""
    
    for i in tqdm(range(len(data))):
        
        right_answer = data['answer'][i]
        premise = data['premise'][i]

        same_ner_tag_answer_array = []
        same_chunking_tag_answer_array = []

        ner_tag_answer = data['ner_tag_answer'][i][0]
        ner_tag_premise = data['ner_tag_premise'][i]

        chunking_tag_answer = data['chunking_tag_answer'][i][0]
        chunking_tag_premise = data['chunking_tag_premise'][i]
        
        # Grouped with the same NER & Chunking group, between answer and word of premise
        data['same_ner_tag_answer'][i] = grouping_same_tag(ner_tag_answer,
                                                           ner_tag_premise,
                                                           same_ner_tag_answer_array)
        
        data['same_chunking_tag_answer'][i] = grouping_same_tag(chunking_tag_answer, 
                                                                chunking_tag_premise, 
                                                                same_chunking_tag_answer_array)
               
        # Start to create wrong answer
        plausible_answer_array = []

        # Perform NER classification
        # If the NER of the right_answer can be detected, then calculate the distance using semantic 
        # similarity or word vectors between the right_answer and various possible wrong_answers with 
        # the same NER as the right_answer. Once done, proceed to the final wrong_answer.
        if data['same_ner_tag_answer'][i] != []:
            wrong_answer, plausible_answer_array = sorting_similarity(data, right_answer, \
                                                                      i, "ner", plausible_answer_array)
            data['properties'][i] = """IDENTICAL NER labels were found, and the highest similarity 
                                    score same NER array was selected"""
            
        # If the NER of the right_answer cannot be detected (NULL) or context/premise does not contain 
        # any of NER of right_answer, then the POS/Chunking of the right_answer will be identified.
        # Perform POS/Chunking classification
        else:
            
            # If the POS/Chunking of the right_answer can be detected, then calculate the distance 
            # using semantic similarity or word vectors between the right_answer and various possible 
            # wrong_answers with the same POS/Chunking as the right_answer. Once done, proceed to the 
            # final wrong_answer.
            if data['same_chunking_tag_answer'][i] != []:
                wrong_answer, plausible_answer_array = sorting_similarity(data, right_answer, \
                                                                          i, "chunking", plausible_answer_array)
                data['properties'][i] = """IDENTICAL Chunking labels were found, and the highest similarity 
                                        score from same Chunking array was selected"""
            
            # If the POS/Chunking of the right_answer cannot be detected (NULL) or context/premise 
            # does not contain any of NER of right_answer, then the final wrong_answer will be chosen 
            # based on a random word (random_word) from the context/premise.
            else:
                for chunking_tag in chunking_tag_premise:
                    plausible_answer_array.append(chunking_tag[1])

                wrong_answer, plausible_answer_array = sorting_similarity(data, right_answer, \
                                                                          i, "none", plausible_answer_array)
                data['properties'][i] = """NO CHUNKING labels were found, and the highest similarity score 
                                        from plausible answer was selected"""

        # Check for preventing same answer for right_answer and wrong_answer  
        right_answer_span = find_substring_span(premise, right_answer)
        wrong_answer_span = find_substring_span(premise, wrong_answer)
        
        is_span_or_same_literal = check_span_overlap(right_answer_span, wrong_answer_span) \
                or check_string_overlap(right_answer.lower(), wrong_answer.lower())

        if is_span_or_same_literal:

            # Removing right answer & wrong answer in this particular time
            wrong_answer, properties, plausible_answer_array = replace_same_answer(right_answer, 
                                                                                  wrong_answer, 
                                                                                  premise, 
                                                                                  plausible_answer_array)
            data['properties'][i] = properties
        
        data['wrong_answer'][i] = wrong_answer
        data['plausible_answer_based_on_method'][i] = plausible_answer_array
            
    return data       

In [28]:
def create_wrong_answer_with_removing_invalid_data(data):
    
    data['same_ner_tag_answer'] = ""
    data['same_chunking_tag_answer'] = ""
    data['wrong_answer'] = ""
    data['plausible_answer_based_on_method'] = ""
    data['properties'] = ""
    
    for i in tqdm(range(len(data))):
        
        right_answer = data['answer'][i]
        premise = data['premise'][i]

        same_ner_tag_answer_array = []
        same_chunking_tag_answer_array = []

        ner_tag_answer = data['ner_tag_answer'][i][0]
        ner_tag_premise = data['ner_tag_premise'][i]

        chunking_tag_answer = data['chunking_tag_answer'][i][0]
        chunking_tag_premise = data['chunking_tag_premise'][i]
        
        # Grouped with the same NER & Chunking group, between answer and word of premise
        data['same_ner_tag_answer'][i] = grouping_same_tag(ner_tag_answer,
                                                           ner_tag_premise,
                                                           same_ner_tag_answer_array)
        
        data['same_chunking_tag_answer'][i] = grouping_same_tag(chunking_tag_answer, 
                                                                chunking_tag_premise, 
                                                                same_chunking_tag_answer_array)
               
        # Start to create wrong answer
        plausible_answer_array = []

        # Perform NER classification
        # If the NER of the right_answer can be detected, then calculate the distance using semantic 
        # similarity or word vectors between the right_answer and various possible wrong_answers with 
        # the same NER as the right_answer. Once done, proceed to the final wrong_answer.
        if data['same_ner_tag_answer'][i] != []:
            wrong_answer, plausible_answer_array = sorting_similarity(data, right_answer, \
                                                                      i, "ner", plausible_answer_array)
            data['properties'][i] = """IDENTICAL NER labels were found, and the highest similarity 
                                    score same NER array was selected"""
            
        # If the NER of the right_answer cannot be detected (NULL) or context/premise does not contain 
        # any of NER of right_answer, then drop that particular row data.
        else:
            data.drop(i, inplace=True)
            data.reset_index(drop=True)
            continue
        
        # Check for preventing same answer for right_answer and wrong_answer  
        right_answer_span = find_substring_span(premise, right_answer)
        wrong_answer_span = find_substring_span(premise, wrong_answer)
        
        is_span_or_same_literal = check_span_overlap(right_answer_span, wrong_answer_span) \
                or check_string_overlap(right_answer.lower(), wrong_answer.lower())

        if is_span_or_same_literal:

            # Removing right answer & wrong answer in this particular time
            wrong_answer, properties, plausible_answer_array = replace_same_answer(right_answer, 
                                                                                  wrong_answer, 
                                                                                  premise, 
                                                                                  plausible_answer_array)
            data['properties'][i] = properties
        
        data['wrong_answer'][i] = wrong_answer
        data['plausible_answer_based_on_method'][i] = plausible_answer_array
            
    return data       

In [29]:
1/0

ZeroDivisionError: division by zero

In [ ]:
x = create_wrong_answer_with_removing_invalid_data(data_nli_train_df)
y = create_wrong_answer_with_removing_invalid_data(data_nli_val_df)
z = create_wrong_answer_with_removing_invalid_data(data_nli_test_df)

In [ ]:
data_nli_train_df = create_wrong_answer(data_nli_train_df)
data_nli_val_df = create_wrong_answer(data_nli_val_df)
data_nli_test_df = create_wrong_answer(data_nli_test_df)

# Split to two dataset: right dataset & wrong dataset

In [ ]:
def move_to_column_number(data, column_name="hypothesis", column_num=3):

    cols = list(data.columns)
    cols.remove(column_name)
    cols.insert(column_num, column_name)

    data = data[cols]
    
    return data

In [ ]:
columns_to_exclude = ['wrong_answer']

data_nli_right_train_df = data_nli_train_df.drop(columns=columns_to_exclude).copy()
data_nli_right_val_df = data_nli_val_df.drop(columns=columns_to_exclude).copy()
data_nli_right_test_df = data_nli_test_df.drop(columns=columns_to_exclude).copy()

In [ ]:
columns_to_exclude = ['answer']

data_nli_wrong_train_df = data_nli_train_df.drop(columns=columns_to_exclude).copy()
data_nli_wrong_val_df = data_nli_val_df.drop(columns=columns_to_exclude).copy()
data_nli_wrong_test_df = data_nli_test_df.drop(columns=columns_to_exclude).copy()

data_nli_wrong_train_df.rename(columns={'wrong_answer': 'answer'}, inplace=True)
data_nli_wrong_val_df.rename(columns={'wrong_answer': 'answer'}, inplace=True)
data_nli_wrong_test_df.rename(columns={'wrong_answer': 'answer'}, inplace=True)

data_nli_wrong_train_df = move_to_column_number(data_nli_wrong_train_df, "answer", 2)
data_nli_wrong_val_df = move_to_column_number(data_nli_wrong_val_df, "answer", 2)
data_nli_wrong_test_df = move_to_column_number(data_nli_wrong_test_df, "answer", 2)

# Convert question-answer pair to hypothesis

In [ ]:
def convert_question_and_answer_to_hypothesis(data):
    for i in range(len(data)):
        data['hypothesis'] = data['question'] + ' ' + data['answer']
    return data

In [ ]:
data_nli_right_train_df = convert_question_and_answer_to_hypothesis(data_nli_right_train_df)
data_nli_right_val_df = convert_question_and_answer_to_hypothesis(data_nli_right_val_df)
data_nli_right_test_df = convert_question_and_answer_to_hypothesis(data_nli_right_test_df)

data_nli_right_train_df = move_to_column_number(data_nli_right_train_df, "hypothesis", 3)
data_nli_right_val_df = move_to_column_number(data_nli_right_val_df, "hypothesis", 3)
data_nli_right_test_df = move_to_column_number(data_nli_right_test_df, "hypothesis", 3)

In [ ]:
data_nli_wrong_train_df = convert_question_and_answer_to_hypothesis(data_nli_wrong_train_df)
data_nli_wrong_val_df = convert_question_and_answer_to_hypothesis(data_nli_wrong_val_df)
data_nli_wrong_test_df = convert_question_and_answer_to_hypothesis(data_nli_wrong_test_df)

data_nli_wrong_train_df = move_to_column_number(data_nli_wrong_train_df, "hypothesis", 3)
data_nli_wrong_val_df = move_to_column_number(data_nli_wrong_val_df, "hypothesis", 3)
data_nli_wrong_test_df = move_to_column_number(data_nli_wrong_test_df, "hypothesis", 3)

# Add label: entailment & contradiction

In [ ]:
data_nli_right_train_df['label'] = 'entailment'
data_nli_right_val_df['label'] = 'entailment'
data_nli_right_test_df['label'] = 'entailment'

data_nli_right_train_df = move_to_column_number(data_nli_right_train_df, "label", 4)
data_nli_right_train_df = move_to_column_number(data_nli_right_val_df, "label", 4)
data_nli_right_train_df = move_to_column_number(data_nli_right_test_df, "label", 4)

In [ ]:
data_nli_wrong_train_df['label'] = 'contradiction'
data_nli_wrong_val_df['label'] = 'contradiction'
data_nli_wrong_test_df['label'] = 'contradiction'

data_nli_wrong_train_df = move_to_column_number(data_nli_wrong_train_df, "label", 4)
data_nli_wrong_val_df = move_to_column_number(data_nli_wrong_val_df, "label", 4)
data_nli_wrong_test_df = move_to_column_number(data_nli_wrong_test_df, "label", 4)

# Concat the right and wrong NLI to one NLI dataset

In [ ]:
data_nli_train_df_final = pd.concat([data_nli_right_train_df, data_nli_wrong_train_df], axis=0, ignore_index=True)
data_nli_val_df_final = pd.concat([data_nli_right_val_df, data_nli_wrong_val_df], axis=0, ignore_index=True)
data_nli_test_df_final = pd.concat([data_nli_right_test_df, data_nli_wrong_test_df], axis=0, ignore_index=True)

# Convert to DataFrame format to CSV

In [ ]:
data_nli_train_df_final.to_csv("data_nli_train_df.csv", index=False)
data_nli_val_df_final.to_csv("data_nli_val_df.csv", index=False)
data_nli_test_df_final.to_csv("data_nli_test_df.csv", index=False)